# Naive Bayes

## Análise de Sentimentos

In [ ]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

**Ler arquivo de dados e conta a quantidade de linhas**

In [ ]:
dataset = pd.read_csv('Tweets_Mg.csv',encoding='utf-8')

In [ ]:
dataset.info()

In [ ]:
# Expante o tamanho de exibição de coluna de texto
pd.set_option('display.max_colwidth', -1)
dataset.Text.head(100)

**Excluir Linhas duplicadas**

In [ ]:
dataset.drop_duplicates(['Text'],inplace=True)

In [ ]:
dataset.count()

**Removendo Colunas desnecessárias**

In [ ]:
for i in dataset.columns.values:
    if i.startswith('Unnamed'):
        dataset.drop(i, axis=1, inplace=True)
        print ('Colunas Deletadas:', i)

In [ ]:
# Visualizando as colunas do dataset
dataset.columns

**Distribuição das classes neutros, positivos e negativos**

In [ ]:
%matplotlib inline
dataset.Classificacao.value_counts().plot(kind='bar')

** Separando tweets e suas classes
**

In [ ]:
tweets = dataset['Text']
classes = dataset['Classificacao']

**Gerando o modelo**

In [ ]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)

In [ ]:
type(freq_tweets)

In [ ]:
freq_tweets.shape

In [ ]:
freq_tweets.A

**Cria um modelo usando Naive Bayes Multinomial**

In [ ]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

** Testando o modelo com algumas instâncias simples**

In [ ]:
# defina instâncias de teste dentro de uma lista
testes = ['Esse governo está no início, vamos ver o que vai dar',
          'Estou muito feliz com o governo de Minas esse ano',
          'O estado de Minas Gerais decretou calamidade financeira!!!',
          'A segurança desse país está deixando a desejar',
          'O governador de Minas é mais uma vez do PT']

In [ ]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(testes)

In [ ]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

In [ ]:
# Probabilidades de cada classe
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

## Avaliação o modelo ##

* Usaremos a técnica de Cross Validation para avaliação dos modelos.
* Esta técnica será usada com 10 folds.

**Fazendo o cross validation do modelo**

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

**Matriz de confusão**

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

** Métricas de validação do modelo**

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

## Tunning do Naive Bayes

In [ ]:
from sklearn.model_selection import GridSearchCV

** Defina uma lista de valores de Alpha**

In [ ]:
lista_alpha = list(range(1,11))
lista_alpha

** Cria um dicionario com o nome do parametro e a lista de valores **

In [ ]:
parametros_grid = dict(alpha=lista_alpha)

**Instancia um modelo**

In [ ]:
nvModelo = MultinomialNB()

** Cria o objeto Grid**

In [ ]:
grid = GridSearchCV(nvModelo, parametros_grid, cv=10, scoring='accuracy')

**Roda o Grid**

In [ ]:
freq_tweets = vectorizer.fit_transform(tweets)
grid.fit(freq_tweets, classes)

** Melhores Scores**

In [ ]:
grid.grid_scores_

** Melhores parâmetros**

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB(alpha=5)
modelo.fit(freq_tweets,classes)

In [ ]:
freq_testes = vectorizer.transform(testes)

In [ ]:
# Fazendo a classificação com o modelo treinado.
modelo.predict(freq_testes)

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

** Considerações Finais**

* Considere aumentar a quantidade de dados de treino.

* Pela sua simplicidade o Naive Bayes pode ser usado perfeitamente como um algoritmo de Baseline.

* Considere alterar os parâmetros do algoritmo.

* Em tarefas de classificação de textos considere utilizar outras modelagens como Bigrams por exemplo.

* Em tarefas envolvendo textos outras técnicas como stemming ou remoção de stoptowords podem dar bons resultados.
